In [ ]:
from selenium import webdriver
import pandas as pd
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException


def next_page(driver):
    try:
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
    except NoSuchElementException:
        print("NoSuchElementException")


def extract_page_data(driver):
    data = []
    houses = driver.find_elements(By.CSS_SELECTOR, 'dl.clearfix')

    for house in houses:
        try:
            title = house.find_element(By.CSS_SELECTOR, "h4.clearfix a[title]").get_attribute("title")
            house_info = house.find_element(By.CSS_SELECTOR, 'p.tel_shop').text
            info_div = [x.strip() for x in house_info.split("|") if x.strip()]
            community = house.find_element(By.CSS_SELECTOR, 'p.add_shop a').get_attribute('title')
            address = house.find_element(By.CSS_SELECTOR, 'p.add_shop span').text
            price = house.find_element(By.CSS_SELECTOR, "dd.price_right")
            total_price = price.find_element(By.CSS_SELECTOR, "b").text.strip()
            unit_price = price.find_elements(By.CSS_SELECTOR, "span")[-1].text.strip()

            item = {
                '标题': title,
                '户型': info_div[0]if len(info_div) > 0 else '',
                '面积': info_div[1]if len(info_div) > 1 else '',
                '层数': info_div[2]if len(info_div) > 2 else '',
                '朝向': info_div[3]if len(info_div) > 3 else '',
                '建成时间': info_div[4]if len(info_div) > 4 else '',
                '小区': community,
                '地址': address,
                '总价（万）': total_price,
                '单位价格': unit_price
            }
            data.append(item)

        except Exception as e:
            print(f'解析房源信息时出错: {e}')
            continue

        return data


def save_data(data, filename, is_first_page=False):
    df = pd.DataFrame(data)

    if is_first_page:
        df.to_csv(filename, mode='w', index=False, encoding='utf_8_sig', header=True)
    else:
        df.to_csv(filename, mode='a', index=False, encoding='utf_8_sig', header=False)


def main():
    driver = webdriver.Edge()
    url = 'https://esf.fang.com/house-a015277-b02314/'
    filename = 'xisanqi_esf_price.csv'
    first_page = True

    driver.get(url)
    sleep(10)

    page_data = extract_page_data(driver)
    save_data(page_data, filename, is_first_page=first_page)
    print("第1页数据处理完成")

    for i in range(2,11):
        next_page(driver)
        sleep(10)
        page_data = scrape_page_data(driver)
        save_data(page_data, filename)
        print(f"第{i}页数据处理完成")
    print("前10页处理完成,已生成csv文件")
    driver.quit()


if __name__ == '__main__':
    main()